In [1]:
include("CGS_POMDP.jl");
pomdp = CGSPOMDP();

### Test: Action function returns all actions

In [3]:
all_actions = POMDPs.actions(pomdp);

### Test: Observation function returns correct distributions

In [4]:
POMDPs.observation(pomdp, all_actions[21], pomdp.state) # Can take up to 7 mins to run!

Distributions.ProductDistribution{2, 1, Vector{FullNormal}, Distributions.Continuous, Float64}(
dists: FullNormal[FullNormal(
dim: 20
μ: [-0.303163597751918, -0.36325826622029944, 0.48468871941794606, -0.9461786179300481, -0.7504959933901902, 0.5002691798031457, 0.24063125129256194, -1.9986176106109215, -2.041340515379154, -1.3696842218355274, 0.704609444064891, 0.10393184896688112, -0.7010338011079825, -1.5975551895330204, -0.11361782195584454, 2.3392441190829056, -0.53415273552389, 2.936970148534619, 1.0935761288341714, 0.312933187477843]
Σ: [1.0e-5 -2.2410821171051474e-31 … -9.960364964911766e-31 2.4900912412279416e-31; -2.2410821171051474e-31 1.0e-5 … 1.1205410585525737e-30 -2.8013526463814343e-31; … ; -9.960364964911766e-31 1.1205410585525737e-30 … 1.0e-5 -1.2450456206139708e-30; 2.4900912412279416e-31 -2.8013526463814343e-31 … -1.2450456206139708e-30 1.0e-5]
)
, FullNormal(
dim: 20
μ: [51.71966932938729, 52.65028737411187, 53.85206918306659, 52.470757037000546, 53.30038771465239,

In [13]:
layer, column, pts = 1, :z, [Point(1, 2), Point(2, 3)]

(1, :z, Point{𝔼{2}, Cartesian2D{NoDatum, Quantity{Float64, 𝐋, Unitful.FreeUnits{(m,), 𝐋, nothing}}}}[Point(x: 1.0 m, y: 2.0 m), Point(x: 2.0 m, y: 3.0 m)])

In [26]:
gtlayer = pomdp.state.earth[layer].gt
data_at_all_wells = gtlayer[Multi([pomdp.collected_locs...]), :]

proc = GaussianProcess(
        SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET), 
        mean(gtlayer[:, column])
        ) # TODO: This is a bit of a hack, we assume we have an okay estimate of mean for a layer
        
grid = CartesianGrid(100, 100)

tgt_geom = Multi([pts...])
tgt_realizations = [rand(proc, grid, data_at_all_wells)[tgt_geom, :] for _ in 1:100];

Z = [tgt_rlz[:, column] for tgt_rlz in tgt_realizations]
Z_matrix = hcat(Z...)
mean_vector = mean(Z_matrix, dims=2) |> vec
cov_matrix = cov(Z_matrix')

@assert all(eigen(cov_matrix).values .> 0)
feat_distr = MvNormal(mean_vector, cov_matrix)


FullNormal(
dim: 2
μ: [0.11533680749170236, 0.15505613903533347]
Σ: [4.085307257123521 3.264314149699929; 3.264314149699929 3.4137896010651327]
)


In [10]:
POMDPs.observation(pomdp, all_actions[2], pomdp.state)

DiagNormal(
dim: 15
μ: [-0.6364280174685096, 167.2321078826396, 64.771635930932, 6.542273917176749, 39.02912312910985, 55.75450774353556, 16.859836249679415, 32.32996730047827, 37.750680260616036, 19.588373847142915, 157.9622153669731, 57.19186844658594, 30.18687893822244, 41.23311842016659, 71.24681057940015]
Σ: [0.6299078215826791 0.0 … 0.0 0.0; 0.0 0.6299078215826791 … 0.0 0.0; … ; 0.0 0.0 … 0.6299078215826791 0.0; 0.0 0.0 … 0.0 0.6299078215826791]
)


### Test: Points close are predicted with more accuracy and less uncertainty than points far.

In [5]:
gtlayer = pomdp.state.earth[1].gt

# seismic_line = (x1=0.0, y1=0.0, x2=5.0, y2=98.0)
well1 = Point(19, 34)
well2 = Point(30, 12)
well_close = Point(31, 14)
well_far = Point(98, 97)

all_wells = Multi([well1, well2])
data_at_all_wells = gtlayer[all_wells, :]

γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET)

okrig = GeoStatsModels.OrdinaryKriging(γ)
fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

# We notice that prediction at close has much less variance than prediction at far, 
# and as an additional sanity check the prediction at close is close to the well2 point.
# Krigging predictprob only does one attribute at a time.

probs_far = GeoStatsModels.predictprob(fitkrig, :z, well_far)
probs_close = GeoStatsModels.predictprob(fitkrig, :z, well_close)
probs_close, probs_far

(Normal{Float64}(μ=1.7700934776094555, σ=1.316834057971783), Normal{Float64}(μ=1.1175853300669871, σ=2.1213203435596424))

### Test: Uncertainty is low around known points.
Additionally, as nugget is increased uncertainty (both globally and at known points) increases.
Pay attention to colorbar when verifying this.

In [6]:
function visualize_uncertainty(all_locs::Multi, layer::Int, col_name::String)
    gtlayer = pomdp.state.earth[layer].gt
    data_at_all_wells = gtlayer[all_locs, :]

    γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET) # Each feature can have a different nugget in the future.
    okrig = GeoStatsModels.OrdinaryKriging(γ)
    fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

    df = copy(pomdp.state.earth[1].df)
    dom = domain(pomdp.state.earth[1].gt)

    var_column = Symbol("$(col_name)_variances")
    column = Symbol("$col_name")
    df[!, var_column] = [GeoStatsModels.predictprob(fitkrig, column, pt).σ
                    for pt in domain(pomdp.state.earth[1].gt)]
    var_map = georef(df, dom)
    viewer(var_map)
end

# visualize_uncertainty(all_wells, 2, "z")

visualize_uncertainty (generic function with 1 method)

### Buy well data at 3-4 locations and visualize belief

In [7]:
buy_well_data(pomdp, 2)
buy_well_data(pomdp, 5)
buy_well_data(pomdp, 7)

4-element Vector{Geometry}:
 Point(x: 18.0 m, y: 89.0 m)
 Point(x: 18.0 m, y: 89.0 m)
 Point(x: 5.0 m, y: 34.0 m)
 Point(x: 94.0 m, y: 78.0 m)

In [8]:
visualize_uncertainty(Multi([pomdp.collected_locs...]), 3, "z") # Note this is time consuming at just 3 gathered points.

In [40]:
distr = observe(pomdp, Point(18, 24), 3, "z")
rand(distr)

20.583216725844274